In [1]:
!ls

final_data.pkl	problems.txt  Untitled.ipynb  users.txt


In [2]:
import numpy as np
import pickle
import pandas as pd
import json
import os
import copy
import gc
import math
import scipy.sparse as sp

In [3]:
f = open("final_data.pkl","rb")

In [4]:
allData = pickle.load(f)

In [5]:
f.close()

In [6]:
print(len(allData))

146976


In [7]:
print(len(allData[0]))

2


In [8]:
print(len(allData[0][1]))

1765


In [9]:
print(len(allData[0][0]))

7


In [10]:
print(allData[0][0])

tourist


In [11]:
print(allData[0][1][1])

{'id': 42104799, 'creationTimeSeconds': 1535237032, 'verdict': 'WRONG_ANSWER', 'passedTestCount': 0, 'problem': {'contestId': 986, 'index': 'D'}}


In [12]:
users = json.load(open("users.txt", "r"))
print(type(users))
users = users['result']
print(type(users))
print((users[0]))

<class 'dict'>
<class 'list'>
{'lastName': 'Korotkevich', 'country': 'Belarus', 'lastOnlineTimeSeconds': 1535792604, 'city': 'Gomel', 'rating': 3434, 'friendOfCount': 14191, 'titlePhoto': '//userpic.codeforces.com/422/title/6dd5ba0df6a2f834.jpg', 'handle': 'tourist', 'avatar': '//userpic.codeforces.com/422/avatar/1d4e692d6695dfa6.jpg', 'firstName': 'Gennady', 'contribution': 158, 'organization': 'ITMO University', 'rank': 'legendary grandmaster', 'maxRating': 3739, 'registrationTimeSeconds': 1265987288, 'maxRank': 'legendary grandmaster'}


In [13]:
# maps the user_handle to user_id
user_id = {}
# gives the user_handle corresponding to the user_id
user_handle = []
for i in range(len(users)):
    user_id[users[i]['handle']] = i
    user_handle.append(users[i]['handle'])

print(len(user_handle))

146976


In [14]:
problems = json.load(open('problems.txt', 'r'))
problems = problems['result']
problems = problems['problems']
print(len(problems))
print(problems[0])

4433
{'contestId': 1029, 'index': 'F', 'name': 'Multicolored Markers', 'type': 'PROGRAMMING', 'tags': ['binary search', 'math', 'number theory']}


In [15]:
# maps the contest_id+index to id
with open('problem_mappings.pkl','rb') as f:
    problem_id = pickle.load(f)
  
# gives the contest_id+index corresponding to the problem_id
problem_name = []
l = len(problems)
for i in range(len(problems)-1, 0, -1):
    problem_name.append(str(problems[i]['contestId']) + problems[i]['index'])

# mappings is used to get the contestIds for the problems not there in problems.txt
# this happens in case of various div2 questions which are also there in div1
with open('plsss.pkl','rb') as f:
    mappings = pickle.load(f)

In [16]:
def get_problem_id(cid, idx):
    if str(cid)+idx in problem_id:
        return problem_id[str(cid)+idx]
    if cid>len(mappings) or len(idx)>1:
        return -1
    if mappings[cid] !=  0:
        #print(str(cid)+idx)
        idx = chr(int(ord(idx) - mappings[cid][1]))
        cid = cid + mappings[cid][0]
        #print('haha'+str(cid)+idx)
        if str(cid) + idx in problem_id:
            return problem_id[str(cid)+idx]
    return -1

In [17]:
problem_AC = np.zeros(len(problems))
problem_attempted = np.zeros(len(problems))
user_AC = np.zeros(147000)
user_attempted = np.zeros(147000)
# change this dtype
mean_user_rating = np.zeros(len(problems), dtype=np.int64)

In [19]:
# initialise to min and max
# dict params: last_sub, first_sub, time_delta, first_AC_sub, user_problem_AC, user_problem_attempted
dt = np.dtype({'names':('last_sub', 'first_sub','first_AC_sub','user_problem_AC','user_problem_attempted'),
                          'formats':(np.int64, np.int64,np.int64,np.int32,np.int32)})
user_prob_matrix = sp.dok_matrix((147000,len(problems)),dtype=dt)
init_data = np.array([(-1,20000000000,20000000000,0,0)],dtype=dt)

In [20]:
for u in range(len(allData)):
    uu = user_id[allData[u][0]]
    rating = users[uu]['rating']
    for s in range(len(allData[u][1])):
        user_submission = allData[u][1][s]
        if 'contestId' not in user_submission['problem'] or 'verdict' not in user_submission:
            continue
        pid = get_problem_id(user_submission['problem']['contestId'],user_submission['problem']['index'])
        if pid == -1:
            continue
        if(user_prob_matrix.get((uu,pid))==0.0):
            user_prob_matrix[uu,pid] = init_data[0]
        creationTimeSeconds = user_submission['creationTimeSeconds']
        user_prob_matrix[uu,pid]['last_sub'] = max(user_prob_matrix[uu,pid]['last_sub'], creationTimeSeconds)
        user_prob_matrix[uu,pid]['first_sub'] = min(user_prob_matrix[uu,pid]['first_sub'], creationTimeSeconds)
        verdict = user_submission['verdict']
        passedTestCount = user_submission['passedTestCount']
        contestId = user_submission['problem']['contestId']
        if verdict == 'OK':
            user_prob_matrix[uu,pid]['first_AC_sub'] = min(user_prob_matrix[uu,pid]['first_AC_sub'], creationTimeSeconds)
            user_prob_matrix[uu,pid]['user_problem_AC'] += 1
            user_AC[uu] += 1
            mean_user_rating[pid] = (mean_user_rating[pid]*problem_AC[pid] + rating)/(problem_AC[pid]+1)
            problem_AC[pid] += 1
        if passedTestCount > 0:
            user_prob_matrix[uu,pid]['user_problem_attempted'] += 1
            user_attempted[uu] += 1
            problem_attempted[pid] += 1
            


/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/scipy/sparse/dok.py:265: FutureWarning: elementwise != comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  if v.ndim == 0 and v != 0:
/home/ubuntu/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.


In [21]:
ans1 = 0
ans2 = 0
for i in range(146976):
  ans1 += user_AC[i]
  
for i in range(len(problems)):
  ans2 += problem_AC[i]
  
print(ans1)
print(ans2)

10632431.0
10632431.0


In [26]:
print(mean_user_rating[900:1000])

[ 775 1134  986 1136 1195 1317 1820 1259 1795 1294 2070 1988  947 1042
 1277 1867 2060 2035 2025  922 1096 1017 1073 1682 1775 1877 1102 1317
 1981 1969 1951  999 1063 1162 1179 1986 2141 2024  829 1696 1726  681
 1650 1670 1925 1238 1750 1743 2048 2026  972 1621 1001 1163 1050 1520
 1972 1135 1153 1836 2113 1991  818 1033 1936 1997 2061 2014 2073 2053
 2159 2257 2112 2005 1730 1903 1994 1769 1727 1729 1690 1863 1615  666
 1467 1459 1024 1300 1846 1949 1922 2150 2117 2067 2038 1956 1417 1489
 1446 2098]


In [27]:
user_prob_matrix_coo = user_prob_matrix.tocoo()

In [32]:
print(user_prob_matrix_coo.shape[1])

4433


In [33]:
sp.save_npz('user_prob_matrix_coo.npz',user_prob_matrix_coo)

In [34]:
np.save('problem_AC.npy',problem_AC)
np.save('problem_attempted.npy',problem_attempted)
np.save('user_AC.npy',user_AC)
np.save('user_attempted.npy',user_attempted)
np.save('mean_user_rating.npy',mean_user_rating)

In [37]:
with open('problem_name.pkl','wb') as f:
    pickle.dump(problem_name,f,protocol=pickle.HIGHEST_PROTOCOL)
# with open('filename.pickle', 'wb') as handle:
#     pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
with open('user_id.pkl','wb') as f:
    pickle.dump(user_id,f,protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
with open('user_handle.pkl','wb') as f:
    pickle.dump(user_handle,f,protocol=pickle.HIGHEST_PROTOCOL)